In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import datetime

# データ生成用モジュール
# クラスと関数だけ集めておくことにする


In [2]:

# ユーティリティ
# 全角→半角
def get_hankaku_from_zenkaku(text):
    return text.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))

# 日付str → datetime
def get_formatted_date(datestr):
    return datetime.datetime.strptime(datestr,"%Y/%m/%d")


In [3]:


#データ整形

# ジャニーズ

class JanniesScraper:                
    
    def __init__(self, datestr, soup_url):
        self.df = pd.DataFrame()
        self.soup =  BeautifulSoup(open(soup_url),"html.parser")
        self.datestr = datestr
    # 公演名
    # str
    def add_name(self):
        kouen_name_list= list(map(lambda x:x.string.strip().replace("\n",""),self.soup.select(".list-ticket-name>p")))
        self.df["name"] = kouen_name_list 
         

    #公演日時
    # Timestamp YYYY/mm/dd
    def add_date(self):
        p = re.compile(r"<[^>]*?>")
        date_list = list( map(\
                             lambda x:
                                    datetime.datetime.strptime( #str→datetime
                                        "20"+ #元の文字が19/05/17で年数に不備が生じるので、先頭に20をつける
                                        p.sub("",str(x)) #タグ除去
                                        .replace(" ","") #空白除去
                                        .replace("\n", "")[:8],"%Y/%m/%d"),#改行除去、曜日を削除、フォーマット変換
                             self.soup.select(".list-ticket-date>span")))
        # type(kouen_data_list[0])
        self.df["date"] = date_list
    
#  　フォーマットユーティリティ
    def format_to_slash(self):
        return self.datestr.replace("-", "/")
    
    #チケットデータ収集日  timestamp YYYY/mm/dd
    def add_collected_at(self):
        slash_date = self.format_to_slash()
        self.df["collected_at"] = datetime.datetime.strptime(slash_date,"%Y/%m/%d")
   
    #公演までの残り日数 int
    def add_left_days(self):
        self.df["left_days"] = (self.df["date"] -self.df["collected_at"]).dt.days  #日時差をint変換
    
    #  ジャニーズダミー 0 1
    def add_is_jannies(self):
        IS_JANNIES = 1
        self.df["is_jannies"] =IS_JANNIES

        
    # 購入可能ダミー　買えるなら1, 買えないなら0　買えるとき →　.list-ticket-order内部にformがあるとき
    def add_available(self):
        list_ticket_order = self.soup.select(".list-ticket-order")
        available = []
        for i, order in enumerate(list_ticket_order):
            if(order.find("form")):
                available.append(1)
            else:
                available.append(0)
        self.df["available"] = available
    

        # 取引あたり価格  int 
    def add_deal_price_list(self):
        deal_price_list = list(map(lambda x:
                          int(re.findall(r"￥\d+,\d+",
                                     str(x))[0][1:]
                                     .replace(",", "")),
                          self.soup.select(".list-ticket-price")))
        self.df["price_per_deal"] = deal_price_list
        
    
    #連番数 int
    def add_unit(self):
        price_list = self.soup.select(".list-ticket-price")
        unit_list = []
        for prc in price_list:
            if "ticket-sellunit" in str(prc):
                unit_list.append(
                    int(prc.select(".ticket-sellunit")[0].get_text()))
            else:
                unit_list.append(1)
        self.df["unit"] = unit_list

#   単位あたり価格 intのつもり
    def add_price_per_unit(self):
        self.df["price_per_unit"] = self.df["price_per_deal"]/ self.df["unit"]

#     取引あたり価格/連番/単位あたり価格の順で ただのファサード
    def add_price_info_bundle(self):
        self.add_deal_price_list()
        self.add_unit() 
        self.add_price_per_unit()
    
    #　チケット転売施行ダミー 01
    # 収集日によって決定　施行後なら1、施行前なら0
    def add_after(self):
        ENFORCE_DATE = "2019-06-14"
        self.df["after"]=   (self.df["collected_at"] >=ENFORCE_DATE)*1
    
        
#  施行前後ダミー×アーティストグループダミーの交差項
    
    def add_cross_term(self):
        self.df["after_and_js_jannies"] = self.df["after"] * self.df["is_jannies"]
        

#         (メインロジック):列の追加から書き出し
    def store_data(self):
        self.add_name() #名前
        self.add_date() #日付
        self.add_collected_at() #収集日
        self.add_left_days() #残り日数
        self.add_is_jannies() #グループダミー
        self.add_available() #購入可能
        self.add_price_info_bundle() #取引価格/枚数/単位価格
        self.add_after() #法施行前後ダミー
        self.add_cross_term() #交差項
        
        #     CSV書き出し
    def write_in_csv(self, filename):
        self.df.to_csv(filename,index=0)
    


In [53]:
# テスト
# TODO ScraperインスタンスにDFとSOUPと日付を保持するよう改修
# コンストラクタにURLと日付

# 生成したCSVをファイル構造を同じくする/CSV/アーティスト名/日付/ticket-data:n.csvに保存
# アーティストごと結合したCSVを /CSV/アーティスト/united-by-artist.csvに保存
# ジャニーズ結合データを/CSV/united-by-jannies.csvに保存
# パスをインスタンス変数にしてwriteも中で行えるようにする

# 演歌グループで同じ構造のクラス生成
# 書き込みまでデータ化
soup_url = "./jannies.html"
sc = JanniesScraper("2019-05-16",soup_url)
sc.store_data()
sc.write_in_csv("./sample.csv")
df = pd.read_csv("sample.csv")
df


,name,date,collected_at,left_days,is_jannies,available,price_per_deal,unit,price_per_unit,after,after_and_js_jannies
0,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,77000,1,77000.0,0,0
1,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,0,99000,1,99000.0,0,0
2,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,0,150000,1,150000.0,0,0
3,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,215000,1,215000.0,0,0
4,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,230000,1,230000.0,0,0
5,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,230000,2,115000.0,0,0
6,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,0,230000,1,230000.0,0,0
7,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,0,250000,1,250000.0,0,0
8,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,280000,2,140000.0,0,0
9,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-18,2019-05-16,2,1,1,58000,2,29000.0,0,0
